In [3]:
import pandas as pd
import numpy as np
#get the smaller portion of data
table = pd.read_csv('processed_data.csv')
subset_table = table.sample(frac=0.3, random_state=42)
utility_matrix = subset_table.pivot_table(index='anime_id', columns='user_id', values='rating')
utility_matrix.fillna(0,inplace=True)
file_path = "smaller_utility_matrix.csv"
utility_matrix.reset_index(inplace=True)
utility_matrix.to_csv(file_path,index=False)


In [6]:
utility_matrix.head(20)

user_id,anime_id,1,2,3,4,5,6,7,8,10,...,73503,73504,73506,73507,73508,73510,73511,73512,73513,73515
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,8.0,0.0,0.0,0.0,0.0,8.0,0.0
2,6,0.0,0.0,0.0,-1.0,8.0,0.0,0.0,0.0,0.0,...,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
